In [23]:
import pandas as pd
import numpy as np
from PIL import Image
import re
import nltk
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from  tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import tensorflow as tf
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from string import punctuation
from gensim.models import KeyedVectors
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
import pandas as pd
import numpy as np
from PIL import Image
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from string import punctuation
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\razak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
df_train = pd.read_json('hateful_memes_expanded/train.jsonl',lines = True , encoding = 'utf-8')

df_test_seen = pd.read_json('hateful_memes_expanded/test_seen.jsonl',lines = True , encoding = 'utf-8')
df_test_unseen = pd.read_json('hateful_memes_expanded/test_unseen.jsonl',lines = True , encoding = 'utf-8')
df_test = df_test_seen.append(df_test_unseen, ignore_index=True)

df_dev_seen = pd.read_json('hateful_memes_expanded/dev_seen.jsonl',lines = True , encoding = 'utf-8')
df_dev_unseen = pd.read_json('hateful_memes_expanded/dev_unseen.jsonl',lines = True , encoding = 'utf-8')
df_val = df_dev_seen.append(df_dev_unseen, ignore_index=True)

In [16]:
print(len(df_train))
print(len(df_test))
print(len(df_val))


12887
3000
1040


In [32]:
df_train

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."
...,...,...,...,...
12882,fyAh3I0.png,img/fyAh3I0.png,0,WHAT IF POKEMON GO WAS RELEASED TO D US FRO TH...
12883,CMQqhImUkAAKeno.jpg,img/CMQqhImUkAAKeno.jpg,0,Donald Trump 's hair looks like someone tried ...
12884,tINjUCc.png,img/tINjUCc.png,0,MAMAS Who am I supposed to vote for ? Am l sup...
12885,eI2N5iQ.png,img/eI2N5iQ.png,0,EcakpnBeHn rnacoBnl 18 minutes ago We will hav...


In [48]:
list_of_pics = list()
text = list()
label = list()
errorfiles = 0 
for row in df_train.iterrows():
    try:
        image=Image.open('hateful_memes_expanded/'+row[1]["img"])
        image=image.resize((200,200))
        list_of_pics.append(np.asarray(image))
        text.append(row[1]["text"])
        label.append(row[1]["label"])
    except FileNotFoundError:
        errorfiles+=1
        pass

X_img_train = np.array(list_of_pics)
X_text_train = np.array(text)
Y_train = np.array(label)
print(errorfiles)

0


In [49]:
list_of_pics = list()
text = list()
label = list()
for row in df_test.iterrows():
    try:
        image=Image.open('hateful_memes_expanded/'+row[1]["img"])
        image=image.resize((200,200))
        list_of_pics.append(np.asarray(image))
        text.append(row[1]["text"])
        label.append(row[1]["label"])
    except FileNotFoundError:
        pass

X_img_test = np.array(list_of_pics)
X_text_test = np.array(text)
Y_test = np.array(label)

In [50]:
list_of_pics = list()
text = list()
label = list()
for row in df_val.iterrows():
    try:
        image=Image.open('hateful_memes_expanded/'+row[1]["img"])
        image=image.resize((200,200))
        list_of_pics.append(np.asarray(image))
        text.append(row[1]["text"])
        label.append(row[1]["label"])
    except FileNotFoundError:
        pass

X_img_val = np.array(list_of_pics)
X_text_val = np.array(text)
Y_val = np.array(label)

In [51]:
print(len(X_img_train))
print(len(X_text_train))
print(len(Y_train))

12887
12887
12887


In [52]:
dataset = dict()

dataset["X_img_val"] = X_img_val
dataset["X_text_val"] = X_text_val
dataset["Y_val"] = Y_val

dataset["X_img_test"] = X_img_test
dataset["X_text_test"] = X_text_test
dataset["Y_test"] = Y_test

dataset["X_img_train"] = X_img_train
dataset["X_text_train"] = X_text_train
dataset["Y_train"] = Y_train



In [53]:
# import pickle
# pickle.dump(dataset, open("dataset.pk", "wb"))

In [54]:
import pickle
dataset = pickle.load(open("dataset.pk", "rb" ))

In [55]:
X_img_val = dataset["X_img_val"]
X_text_val = dataset["X_text_val"]
Y_val = dataset["Y_val"] 

X_img_test = dataset["X_img_test"]
X_text_test = dataset["X_text_test"]
Y_test = dataset["Y_test"]

X_img_train = dataset["X_img_train"]
X_text_train = dataset["X_text_train"]
Y_train = dataset["Y_train"]

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from  tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [58]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = pd.Series(X_text_train).map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_text_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 20241 unique tokens.
Max len: 1601


In [74]:
word2vecDict = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
embed_size = 300
embeddings_index = dict()
for word in word2vecDict.key_to_index:
  embeddings_index[word] = word2vecDict.word_vec(word)
print("Loaded " + str(len(embeddings_index)) + " word vectors.")
      
embedding_matrix = 1 * np.random.randn(len(word_index)+1, embed_size)

embeddedCount = 0
for word, i in word_index.items():
  i-=1
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None: 
    embedding_matrix[i] = embedding_vector
    embeddedCount+=1
print("total embedded:", embeddedCount, "common words")
      
del(embeddings_index)


C:\Users\razak\AppData\Local\Temp/ipykernel_8260/2712980230.py:5: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embeddings_index[word] = word2vecDict.word_vec(word)


Loaded 3000000 word vectors.
total embedded: 12929 common words


In [75]:
word2vec_embedding_matrix = embedding_matrix

In [76]:
X_text_train = tokenizer.texts_to_sequences(X_text_train)
X_text_train = pad_sequences(X_text_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train tensor:', X_text_train.shape)

Shape of train tensor: (12887, 1601)


In [77]:
X_text_test = tokenizer.texts_to_sequences(X_text_test)
X_text_test = pad_sequences(X_text_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train tensor:', X_text_test.shape)

Shape of train tensor: (3000, 1601)


In [78]:
X_text_val = tokenizer.texts_to_sequences(X_text_val)
X_text_val = pad_sequences(X_text_val, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train tensor:', X_text_val.shape)

Shape of train tensor: (1040, 1601)


In [82]:
lstm_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], mask_zero=True, input_length=MAX_SEQUENCE_LENGTH, trainable=False)(lstm_input)
x = Dropout(0.3)(x)
x = LSTM(64, return_sequences = True)(x)
x = Dropout(0.3)(x)
x = LSTM(64)(x)
x = Dropout(0.3)(x)
lstm_out = Dense(18, activation = 'relu')(x)

print(X_img_train.shape[1], X_img_train.shape[2], X_img_train.shape[3])
cnn_input = Input(shape=(X_img_train.shape[1], X_img_train.shape[2], X_img_train.shape[3]))
y = Conv2D(32, (3, 3), activation='relu', input_shape=(X_img_train.shape[1], X_img_train.shape[2], X_img_train.shape[3]))(cnn_input)
y = MaxPooling2D(2, 2)(y)
y = Conv2D(64, (3, 3), activation='relu')(y)
y = MaxPooling2D(2, 2)(y)
y = Conv2D(128, (3, 3), activation='relu')(y)
y = MaxPooling2D(2, 2)(y)
y = Conv2D(128, (3, 3), activation='relu')(y)
y = MaxPooling2D(2, 2)(y)
y = Flatten()(y)
y = Dropout(0.3)(y)
cnn_out = Dense(512, activation='relu')(y)

concat_inp = concatenate([cnn_out, lstm_out])
z = Dense(256, activation='relu')(concat_inp)
z = Dropout(0.3)(z)
z = Dense(128, activation='relu')(z)
z = Dropout(0.3)(z)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[cnn_input, lstm_input], outputs=[output])
adam = Adam(learning_rate=0.001, decay=1e-5)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


200 200 3


In [84]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 198, 198, 32) 896         input_8[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 99, 99, 32)   0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 97, 97, 64)   18496       max_pooling2d_12[0][0]           
____________________________________________________________________________________________

In [85]:
es = EarlyStopping(patience=5)
csv_logger = CSVLogger('model_history_log_model_lstm.csv', append=True)
check_point = ModelCheckpoint(filepath='best_lstm_model_todate', save_best_only=True, save_weights_only=True)

In [88]:
multimodal_model = model.fit([X_img_train, X_text_train], Y_train, batch_size=256, epochs=20,
                            validation_data=([X_img_val, X_text_val], Y_val), callbacks = [es, csv_logger, check_point], verbose=1)

Epoch 1/20
51/51 [==============================] - 11494s 223s/step - loss: 2.8514 - accuracy: 0.5269 - val_loss: 0.7034 - val_accuracy: 0.4394


PermissionDeniedError: Failed to create a directory: /; Permission denied [Op:MergeV2Checkpoints]